# 1. 加载依赖包


In [34]:
import thulac
import os
import re
import pandas as pd
from gensim.models import Word2Vec

# 2. 读取停词表


In [36]:

def load_stop_word():
    stop_word = ['的', '。', '是', ' ', '\n', '日', '月', '.', '%']
    stop_word_file = 'c:/data/stopwords'
    stop_word_dirs = os.listdir(stop_word_file)
    for textName in stop_word_dirs:
        filename = stop_word_file + '/' + textName
        with open(filename, 'r', encoding='utf-8') as file_object:
            line = file_object.readline()
            while line:
                stop_word.append(line[0:-1])
                line = file_object.readline()
    return stop_word

stop_word = load_stop_word()
print(len(stop_word))
print(stop_word[0:100])

3907
['的', '。', '是', ' ', '\n', '日', '月', '.', '%', '--', '?', '“', '”', '》', '－－', 'able', 'about', 'above', 'according', 'accordingly', 'across', 'actually', 'after', 'afterwards', 'again', 'against', "ain't", 'all', 'allow', 'allows', 'almost', 'alone', 'along', 'already', 'also', 'although', 'always', 'am', 'among', 'amongst', 'an', 'and', 'another', 'any', 'anybody', 'anyhow', 'anyone', 'anything', 'anyway', 'anyways', 'anywhere', 'apart', 'appear', 'appreciate', 'appropriate', 'are', "aren't", 'around', 'as', "a's", 'aside', 'ask', 'asking', 'associated', 'at', 'available', 'away', 'awfully', 'be', 'became', 'because', 'become', 'becomes', 'becoming', 'been', 'before', 'beforehand', 'behind', 'being', 'believe', 'below', 'beside', 'besides', 'best', 'better', 'between', 'beyond', 'both', 'brief', 'but', 'by', 'came', 'can', 'cannot', 'cant', "can't", 'cause', 'causes', 'certain', 'certainly']


# 2.5 分词前修改thulac源码中的bug

对thulac的2个bug进行修改



In [37]:
from contextlib import contextmanager
from io import StringIO
import time


@contextmanager
def add_clock_method_to_time():
    py_gt_3_8 = not hasattr(time, "clock")
    if py_gt_3_8:
        setattr(time, "clock", time.perf_counter)
    try:
        yield
    finally:
        if py_gt_3_8:
            delattr(time, "clock")

# 3. 分词

输入一个文本 对文本中的词语 按照 空格 切分 去掉标点等停词
填充一个全局的字典， 词语 -> 词性


In [55]:
 # 默认模式，分词的同时进行词性标注
thu = thulac.thulac(filt=True, seg_only=False)
word_type = {}
def cut_doc(doc):
    with add_clock_method_to_time():
        words = thu.cut(doc)
    words_in_doc = ""
    for word, wordType in words:
        if word in stop_word: continue
        words_in_doc +=  word + " "
        word_type[word] = wordType
        
    return words_in_doc

words_in_doc = cut_doc("用来控制训练的文章数量")
words_in_doc

word_type

Model loaded succeed


{'控制': 'v', '训练': 'v', '文章': 'n', '数量': 'n'}

# 4. 读取数据

每类新闻中抽取1000个新闻构建 dataframe

In [56]:
my_df = pd.DataFrame([],index=[],columns=['text','label'])

label_num = {}

# 返回 list[str]
def read_content():
    # doc_data 记录 文章数据
    
    # my_df = pd.DataFrame([],index=[],columns=['text','label'])
    
    DOC_COUNT_MAX = 10
    
    
    news_file = 'c:/data/THUCNews'
    news_file_dirs = os.listdir(news_file)
    print(news_file_dirs)
    
    for label in news_file_dirs:
        file = news_file + '/' + label
        doc_ids = os.listdir(file)
        doc_num = 0
        for doc_name in doc_ids:
            doc_num += 1
            if doc_num >= DOC_COUNT_MAX:
                break
            doc_file_name = file + '/' + doc_name
            doc_id = doc_name[0:-4]
            
            doc_content = ""
            
            with open(doc_file_name, 'r', encoding='utf-8') as file_object:
                line = file_object.readline()
                while line:
                    doc_content += line
                    line = file_object.readline()
                    
            # print(doc_name[0:-4], len(doc_content),label)
            my_df.loc[doc_id] = [cut_doc(doc_content), label]
            
           
    return my_df

my_df = read_content()
my_df

['体育', '娱乐', '家居', '彩票', '房产', '教育', '时尚', '时政', '星座', '游戏', '社会', '科技', '股票', '财经']
0.txt 99999.txt
131604.txt 224235.txt
224236.txt 256821.txt
256822.txt 264409.txt
264410.txt 284459.txt
284460.txt 326395.txt
326396.txt 339763.txt
339764.txt 402849.txt
402850.txt 406427.txt
406428.txt 430800.txt
430801.txt 481649.txt
481650.txt 644578.txt
644579.txt 798976.txt
798977.txt 836074.txt


,text,label
0,马晓旭 意外 受伤 警惕 大雨 青睐 殷 家 军 记者 傅亚雨 沈阳 报道 沈阳 摆脱 雨水...,体育
1,商瑞华 首战 复仇 心切 中国 玫瑰 美国 方式 攻克 瑞典 曼 瑞典 商瑞华 求 信心...,体育
10,冠军 球队 迎新 欢乐派 黄旭 获 大奖 张军 赢 PK赛 新 浪 体育 冠军 高尔...,体育
100,辽足 签约 危机 引 注册 难关 高层 威逼利诱 合同 笑里藏刀 新 浪 体育 爆发...,体育
1000,揭秘 谢亚龙 带走 总局 电话 骗 局 复制 杨 轨迹 体坛 周报 特约 记者 张锐 北...,体育
...,...,...
798981,亚洲 燃料油 裂解 价 差 收 窄 跨 合约 遭 大单 抛售 文华 财经 编辑 整理 何丽丽...,财经
798982,艺术品 秋拍 上演 中国 艺术品 市场 经历 低潮 反弹 强势 上涨 北京 荣宝 拉开 拍 ...,财经
798983,欧佩克 会议 幅度 减产 新华网 阿尔及尔 电 记者 郑斌 尔及利亚 新闻社 报道 石油...,财经
798984,中国 棉花 产量 预计 减少 新浪 财经 下午 消息 网站 消息 中国 储备棉 管理 总...,财经


In [58]:
%matplotlib  inline
train_df = my_df
train_df['text_len'] = train_df['text'].apply(lambda x: len(x.split(' ')))
print(train_df['text_len'].describe())

count     126.000000
mean      259.325397
std       250.947479
min         8.000000
25%        91.000000
50%       184.000000
75%       354.250000
max      1572.000000
Name: text_len, dtype: float64
